In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### particle vertices, faces, and insphere radius

In [2]:
anneal = gsd.hoomd.open('./DATA/anneal.gsd')[-1]
verts = anneal.particles.type_shapes[0]['vertices']
particle = coxeter.shapes.ConvexPolyhedron(verts)
faces = particle.faces
sigma = 2*particle.insphere_from_center.radius

#### Defining parameters

In [3]:
# integration timestep
dt=0.0005

# coupling
kT=1.0
tau=100*dt

# reduced pressure
p_init=anneal.log['md/compute/ThermodynamicQuantities/pressure'][0]
p_second=16*kT/particle.volume
tauS=1000*dt

print(p_init, p_second)

0.14516696446769212 15.999984554761161


#### Defining simulation

In [4]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=9512)
sim.create_state_from_gsd(filename='./DATA/anneal.gsd', frame=-1)

#### Defining potential

In [5]:
nl = hoomd.md.nlist.Cell(buffer=0.4)

alj = hoomd.md.pair.aniso.ALJ(nl)
alj.r_cut[('A', 'A')] = 2*particle.circumsphere_from_center.radius + 0.15*sigma
alj.params[('A', 'A')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma, 
                              alpha=0)
alj.shape['A'] = dict(vertices=verts, 
                      faces=faces, 
                      rounding_radii=0) # rounding_radii=0.15*(sigma/2)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/md/pair/aniso.py:589: FutureWarning: 'mode' key is deprectated and will be removed in hoomd 4.0.
  warnings.warn(


#### Defining thermodynamic ensemble

In [6]:
t_ramp=int(1e4)

S = hoomd.variant.Power(A=p_init, 
                        B=p_second,
                        power=5,
                        t_start=sim.timestep, 
                        t_ramp=t_ramp)

npt = hoomd.md.methods.NPT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau, 
                           S=S, 
                           tauS=tauS,
                           couple='xyz')

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/md/methods/methods.py:365: FutureWarning: NPT is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantPressure method with the desired thermostat from hoomd.md.methods.thermostats.
  warnings.warn(


#### Defining integrator

In [7]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[npt],
                                 forces=[alj],
                                 integrate_rotational_dof=True)

#### Defining writer

In [8]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

In [9]:
logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(alj, quantities=['type_shapes'])
logger.add(thermodynamic_properties)

gsd_writer = hoomd.write.GSD(filename='./DATA/compress.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             log=logger,
                             mode='wb')

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x140fc12e0> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


In [10]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### Run simulation

In [11]:
sim.run(0)
# npt.thermalize_thermostat_and_barostat_dof()
alj.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0.0825128,
  'vertices': [[0, 0, 0.981642],
   [0.766908, 0, 0],
   [0.383454, 0.664162, 0],
   [-0.383454, 0.664162, 0],
   [-0.766908, 9.39191e-17, 0],
   [-0.383454, -0.664162, 0],
   [0.383454, -0.664162, 0],
   [0, 0, -0.981642]]}]

In [12]:
sim.run(t_ramp)

In [13]:
phi = particle.volume*sim.state.N_particles/sim.state.box.volume
print('Final packing fraction:', phi)

Final packing fraction: 0.41187389105761474
